# Pramana Explorer: Interactive Navya-Nyaya Reasoning Demo

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/SharathSPhD/pramana/blob/main/notebooks/01_pramana_explorer.ipynb#slideshowMode=true)

This notebook provides an interactive exploration of the Pramana epistemic reasoning engine, demonstrating the 6-phase Navya-Nyaya methodology for systematic logical problem-solving.

## 1. Setup & Configuration

In [ ]:
#@title Setup: Clone repo, install deps, detect GPU (double-click to show code)
import os, sys, subprocess
from pathlib import Path
from IPython.display import display, HTML

# ---- Clone repo (skip if already cloned or running locally) ----
REPO_URL = "https://github.com/SharathSPhD/pramana.git"
REPO_DIR = "pramana"

try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    if not Path(REPO_DIR).exists():
        print("Cloning repository...")
        subprocess.run(["git", "clone", REPO_URL], check=True, capture_output=True)
        print("  Done.")
    os.chdir(REPO_DIR)
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", "-r", "notebooks/requirements.txt"],
                   check=True, capture_output=True)
    sys.path.insert(0, "notebooks")
else:
    # Local: pramana_backend.py should be in the same dir or parent
    if Path("pramana_backend.py").exists():
        sys.path.insert(0, ".")
    elif Path("notebooks/pramana_backend.py").exists():
        sys.path.insert(0, "notebooks")
    else:
        for p in [Path(".."), Path("../notebooks")]:
            if (p / "pramana_backend.py").exists():
                sys.path.insert(0, str(p.resolve()))
                break

# ---- GPU Detection ----
def check_backend():
    """Detect GPU/CPU and display status banner."""
    gpu_name, gpu_mem = None, None
    try:
        result = subprocess.run(
            ["nvidia-smi", "--query-gpu=name,memory.total", "--format=csv,noheader,nounits"],
            capture_output=True, text=True, timeout=5,
        )
        if result.returncode == 0 and result.stdout.strip():
            parts = result.stdout.strip().split(", ")
            gpu_name = parts[0]
            gpu_mem = parts[1] if len(parts) > 1 else "?"
    except Exception:
        pass

    if gpu_name:
        hw = f"<b>GPU: {gpu_name} ({gpu_mem} MB)</b>"
        color = "green"
    else:
        hw = "<b>CPU</b>"
        color = "orange"

    env = "Google Colab" if IN_COLAB else "Local"
    banner = (
        f"<div style='padding:10px;border-radius:8px;border:2px solid {color};margin:8px 0'>"
        f"<span style='font-size:1.2em'>Runtime: <span style='color:{color}'>{hw}</span></span>"
        f"<br>Environment: {env}"
    )
    if not gpu_name:
        banner += "<br><i>For better performance: Runtime -> Change runtime type -> GPU</i>"
    banner += "</div>"
    display(HTML(banner))
    return gpu_name is not None

GPU_AVAILABLE = check_backend()
print(f"Working directory: {os.getcwd()}")

In [ ]:
#@title Imports (double-click to show code)
import os
import re
import json
from pathlib import Path
from typing import Dict, List, Optional, Tuple
from IPython.display import display, HTML, Markdown
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, HBox, VBox, Output

from pramana_backend import (
    create_backend,
    STAGE_CONFIGS,
    OLLAMA_MODEL_MAP,
    build_user_prompt,
    EXAMPLE_PROBLEMS,
    load_test_problems,
    parse_nyaya_phases,
    validate_structure as backend_validate_structure,
    score_content_quality as backend_score_content_quality,
    extract_final_answer,
    score_answers,
    setup_ollama,
    setup_ollama_stage,
    download_gguf,
)

print("\u2713 All modules imported")

In [ ]:
#@title Configuration: Backend selection and model setup (double-click to show code)
# Backends: transformers (local GPU), ollama (local server), llama.cpp (GGUF)
_gpu_label = "Colab GPU" if IN_COLAB else "Local GPU/CPU"
backend_dropdown = widgets.Dropdown(
    options=[
        (f"Transformers ({_gpu_label})", "transformers"),
        (f"Ollama ({_gpu_label})", "ollama"),
        (f"llama.cpp GGUF ({_gpu_label})", "llamacpp"),
    ],
    value="ollama" if IN_COLAB else "transformers",
    description="Backend:",
    style={"description_width": "initial"},
)

stage_dropdown = widgets.Dropdown(
    options=list(STAGE_CONFIGS.keys()),
    value="Stage 0",
    description="Stage:",
    style={"description_width": "initial"},
)

model_variant_dropdown = widgets.Dropdown(
    options=[("Tuned model", "tuned"), ("Base model", "base"), ("Both (comparison)", "both")],
    value="both",
    description="Model:",
    style={"description_width": "initial"},
)

# System prompt -- editable, auto-populated from stage config
_initial_stage = STAGE_CONFIGS["Stage 0"]
system_prompt_textarea = widgets.Textarea(
    value=_initial_stage.system_prompt,
    description="System Prompt:",
    style={"description_width": "initial"},
    layout=widgets.Layout(width="95%", height="120px"),
)

# ---------- Ollama backend-specific settings ----------
_stage0_map = OLLAMA_MODEL_MAP["Stage 0"]
ollama_base_model_input = widgets.Text(
    value=_stage0_map["base"],
    placeholder="Ollama base model tag",
    description="Base model:",
    style={"description_width": "initial"},
    layout=widgets.Layout(width="95%"),
)
ollama_tuned_model_input = widgets.Text(
    value=_stage0_map["tuned"],
    placeholder="Ollama tuned model tag",
    description="Tuned model:",
    style={"description_width": "initial"},
    layout=widgets.Layout(width="95%"),
)
ollama_url_input = widgets.Text(
    value="http://localhost:11434",
    description="Ollama URL:",
    style={"description_width": "initial"},
    layout=widgets.Layout(width="95%"),
)

# llama.cpp
gguf_path_input = widgets.Text(
    value="",
    placeholder="Path to .gguf file (or llama.cpp server URL)",
    description="GGUF path/URL:",
    style={"description_width": "initial"},
    layout=widgets.Layout(width="95%"),
)

# ---------- Stage observer ----------
def _on_stage_change(change):
    stage = change["new"]
    stage_cfg = STAGE_CONFIGS[stage]
    system_prompt_textarea.value = stage_cfg.system_prompt
    if stage in OLLAMA_MODEL_MAP:
        ollama_base_model_input.value = OLLAMA_MODEL_MAP[stage]["base"]
        ollama_tuned_model_input.value = OLLAMA_MODEL_MAP[stage]["tuned"]

stage_dropdown.observe(_on_stage_change, names="value")

# ---------- Backend settings panel ----------
backend_settings_output = Output()

def update_backend_settings(change=None):
    with backend_settings_output:
        backend_settings_output.clear_output()
        bt = backend_dropdown.value
        if bt == "transformers":
            display(widgets.HTML("<i>Uses HuggingFace model IDs from stage config. Requires GPU.</i>"))
        elif bt == "ollama":
            ollama_setup_btn = widgets.Button(description="Auto-Setup Ollama (base + tuned)", button_style="info", icon="magic")
            ollama_setup_out = Output()
            def _ollama_auto(b):
                with ollama_setup_out:
                    ollama_setup_out.clear_output()
                    try:
                        stage = stage_dropdown.value
                        print(f"Setting up Ollama models for {stage}...")
                        result = setup_ollama_stage(stage)
                        ollama_base_model_input.value = result["base"] or ollama_base_model_input.value
                        if result["tuned"]:
                            ollama_tuned_model_input.value = result["tuned"]
                        print(f"\n\u2713 Setup complete for {stage}")
                        print(f"  Base:  {result['base']}")
                        print(f"  Tuned: {result['tuned'] or '(not available)'}")
                    except Exception as e:
                        print(f"\u2717 Setup failed: {e}")
                        import traceback; traceback.print_exc()
            ollama_setup_btn.on_click(_ollama_auto)
            display(VBox([
                widgets.HTML(
                    f"<b>Ollama Settings</b> \u2014 "
                    "Click <b>Auto-Setup</b> to install Ollama + pull/create models for the selected stage."
                ),
                ollama_base_model_input,
                ollama_tuned_model_input,
                ollama_url_input,
                ollama_setup_btn,
                ollama_setup_out,
            ]))
        elif bt == "llamacpp":
            gguf_setup_btn = widgets.Button(description="Auto-Download GGUF", button_style="info", icon="download")
            gguf_setup_out = Output()
            def _gguf_auto(b):
                with gguf_setup_out:
                    gguf_setup_out.clear_output()
                    try:
                        path = download_gguf()
                        gguf_path_input.value = path
                        print(f"\u2713 GGUF downloaded: {path}")
                    except Exception as e:
                        print(f"\u2717 Download failed: {e}")
            gguf_setup_btn.on_click(_gguf_auto)
            display(VBox([
                widgets.HTML("<b>llama.cpp Settings</b> \u2014 Click Auto-Download or provide path to GGUF"),
                gguf_path_input,
                gguf_setup_btn,
                gguf_setup_out,
            ]))

backend_dropdown.observe(update_backend_settings, names='value')

# ---------- Configuration apply ----------
def setup_config():
    global config
    backend_type = backend_dropdown.value
    stage_name = stage_dropdown.value
    variant = model_variant_dropdown.value
    stage_config = STAGE_CONFIGS[stage_name]

    def _make_backend(model_id, role="tuned"):
        if backend_type == "transformers":
            return create_backend("transformers", model_id=model_id)
        elif backend_type == "ollama":
            if role == "base":
                model_name = ollama_base_model_input.value.strip()
            else:
                model_name = ollama_tuned_model_input.value.strip()
            return create_backend("ollama", model_name=model_name,
                                  base_url=ollama_url_input.value.strip())
        elif backend_type == "llamacpp":
            path_or_url = gguf_path_input.value.strip()
            if path_or_url.startswith("http"):
                return create_backend("llamacpp", server_url=path_or_url)
            else:
                return create_backend("llamacpp", model_path=path_or_url)

    try:
        base_backend = _make_backend(stage_config.base_model_id, role="base") if variant in ("base", "both") else None
        tuned_backend = _make_backend(stage_config.tuned_model_id, role="tuned") if variant in ("tuned", "both") else None

        if backend_type == "ollama":
            base_label = ollama_base_model_input.value.strip() if base_backend else None
            tuned_label = ollama_tuned_model_input.value.strip() if tuned_backend else None
        else:
            base_label = stage_config.base_model_id if base_backend else None
            tuned_label = stage_config.tuned_model_id if tuned_backend else None

        active_system_prompt = system_prompt_textarea.value.strip() or stage_config.system_prompt

        config = {
            "backend_type": backend_type,
            "stage_config": stage_config,
            "base_backend": base_backend,
            "tuned_backend": tuned_backend,
            "base_label": base_label,
            "tuned_label": tuned_label,
            "system_prompt": active_system_prompt,
        }
        print(f"\u2713 Configuration set: {stage_name} with {backend_type} backend")
        if base_backend:
            print(f"  Base model ready: {base_label}")
        if tuned_backend:
            print(f"  Tuned model ready: {tuned_label}")
        return config
    except Exception as e:
        print(f"\u2717 Configuration failed: {e}")
        import traceback
        traceback.print_exc()
        return None

setup_button = widgets.Button(description="Apply Configuration", button_style="primary", icon="check")
config_output = Output()

def on_setup_click(button):
    with config_output:
        config_output.clear_output()
        setup_config()

setup_button.on_click(on_setup_click)
update_backend_settings()

display(VBox([
    widgets.HTML("<h3>Configuration</h3>"),
    backend_dropdown,
    stage_dropdown,
    model_variant_dropdown,
    widgets.HTML("<b>System Prompt</b> (auto-populated from stage; editable):"),
    system_prompt_textarea,
    backend_settings_output,
    setup_button,
    config_output,
]))

config = None

## 2. Introduction to Navya-Nyaya Reasoning

### What is Navya-Nyaya?

Navya-Nyaya ("New Logic") is a 2,500-year-old Indian epistemological system that provides a structured methodology for systematic reasoning. Unlike Western formal logic, Navya-Nyaya integrates logic and epistemology, requiring explicit grounding in concrete examples and universal rules.

### The 6-Phase Framework

Pramana enforces a structured 6-phase Nyaya methodology:

1. **Samshaya (Doubt Analysis)** - Classify the type of uncertainty/ambiguity
2. **Pramana (Evidence Sources)** - Identify valid knowledge sources:
   - Pratyaksha (Direct Perception)
   - Anumana (Inference)
   - Upamana (Comparison)
   - Shabda (Testimony)
3. **Pancha Avayava (5-Member Syllogism)** - Construct formal argument with:
   - Pratijna (Thesis)
   - Hetu (Reason)
   - Udaharana (Universal Example)
   - Upanaya (Application)
   - Nigamana (Conclusion)
4. **Tarka (Counterfactual Testing)** - Use reductio ad absurdum to verify conclusions
5. **Hetvabhasa (Fallacy Detection)** - Check for reasoning errors
6. **Nirnaya (Ascertainment)** - Reach definitive conclusion or explicitly state insufficient evidence

In [5]:
#@title Load example from embedded data (no external files needed) (double-click to show code)
# Load example from embedded data (no external files needed)
example = EXAMPLE_PROBLEMS[0]  # pramana-001: constraint satisfaction

display(Markdown(f"## Example Problem: {example['id']}\n\n{example['problem']}"))
display(Markdown(f"**Ground Truth:** {example['ground_truth']}"))
display(Markdown(f"**Problem Type:** {example['problem_type']} | **Difficulty:** {example['difficulty']}"))

## Example Problem: pramana-001

Three people (Alice, Bob, Carol) each have one pet: a cat, a dog, or a fish.

**Constraints**:
1. Alice does not have the cat
2. Bob has the dog
3. Carol does not have the fish

**Question**: Who has which pet?

**Ground Truth:** Alice has the fish, Bob has the dog, Carol has the cat

**Problem Type:** constraint_satisfaction | **Difficulty:** medium

### Phase 1: Samshaya (Doubt Analysis)

The example demonstrates **Samana Dharma Upapatti** (Multiple possibilities share similar properties):

- There are three people and three pets, creating multiple possible assignments
- Without systematic reasoning, we cannot determine which person has which pet
- The doubt arises because multiple arrangements are conceivable

### Phase 2: Pramana (Sources of Knowledge)

The example identifies:

- **Pratyaksha**: Directly stated constraints ("Alice does not have the cat", "Bob has the dog", etc.)
- **Anumana**: Inferences like "If Bob has the dog, then neither Alice nor Carol has the dog"
- **Upamana**: Comparison to constraint satisfaction problems with mutual exclusivity
- **Shabda**: Logical principles (Law of Excluded Middle, Non-Contradiction, etc.)

### Phase 3: Pancha Avayava (5-Member Syllogism)

The example constructs three syllogisms:

1. **Establishing Bob's Pet**: Pratijna="Bob has the dog", supported by direct constraint
2. **Establishing Alice's Pet**: Pratijna="Alice has the fish", via elimination (cannot have cat or dog)
3. **Establishing Carol's Pet**: Pratijna="Carol has the cat", via completeness principle

Each syllogism includes all 5 members with explicit Udaharana (universal rules) and Upanaya (application).

### Phase 4: Tarka (Counterfactual Testing)

The example uses reductio ad absurdum:

- **Hypothesis**: "Suppose Carol does not have the cat"
- **Consequence**: This leads to Carol having no pet, violating completeness
- **Analysis**: This is absurd given the problem constraints
- **Resolution**: Therefore, Carol must have the cat

### Phase 5: Hetvabhasa (Fallacy Check)

The example checks for:

- **Savyabhichara** (Erratic reasoning): None detected
- **Viruddha** (Contradictory reasoning): None detected
- **Prakaranasama** (Circular reasoning): None detected
- **Sadhyasama** (Begging the question): None detected

### Phase 6: Nirnaya (Ascertainment)

**Status**: Definitive Knowledge

**Final Answer**: Alice has the fish, Bob has the dog, and Carol has the cat.

**Confidence**: High - The solution is logically necessary given the constraints.

## 3. Interactive Comparison: Base vs Tuned

In [6]:
#@title Load example problems (self-contained, no external files needed) (double-click to show code)
# Load example problems (self-contained, no external files needed)
examples = load_test_problems("embedded")
print(f"✓ Loaded {len(examples)} embedded examples")

# Display problem list
for i, ex in enumerate(examples):
    print(f"  {i+1}. [{ex['id']}] {ex['problem_type']} ({ex['difficulty']})")

✓ Loaded 5 embedded examples
  1. [pramana-001] constraint_satisfaction (medium)
  2. [pramana-003] transitive_reasoning (medium)
  3. [pramana-005] multi_step_deduction (medium)
  4. [test-001] constraint_satisfaction (easy)
  5. [test-004] boolean_sat (medium)


In [ ]:
#@title Example selector and generation controls (double-click to show code)
# Example selector and generation controls
example_selector = widgets.Dropdown(
    options=[(ex["id"], idx) for idx, ex in enumerate(examples)],
    description="Example:",
    style={"description_width": "initial"},
)

# --- Example preview area ---
example_preview_output = Output()

def _on_example_select(change):
    """Display the selected problem text and ground truth."""
    with example_preview_output:
        example_preview_output.clear_output()
        idx = change["new"]
        ex = examples[idx]
        display(HTML(f"""
        <div style="border:1px solid #ddd; padding:12px; border-radius:6px; background:#f9f9f9; margin:4px 0;">
            <b>Problem:</b>
            <pre style="white-space: pre-wrap; margin:6px 0;">{ex["problem"]}</pre>
            <b>Expected Answer:</b> <code>{ex.get("expected_answer", "N/A")}</code>
        </div>
        """))

example_selector.observe(_on_example_select, names="value")
# Show initial selection immediately
_on_example_select({"new": example_selector.value})

# --- Hyperparameter controls (matching HF Space app) ---
max_tokens_slider = widgets.IntSlider(
    value=2048,
    min=64,
    max=4096,
    step=32,
    description="Max new tokens:",
    style={"description_width": "initial"},
    layout=widgets.Layout(width="95%"),
)

temperature_slider = widgets.FloatSlider(
    value=0.5,
    min=0.0,
    max=1.5,
    step=0.05,
    description="Temperature:",
    style={"description_width": "initial"},
    layout=widgets.Layout(width="95%"),
)

top_p_slider = widgets.FloatSlider(
    value=0.75,
    min=0.0,
    max=1.0,
    step=0.05,
    description="Top-p:",
    style={"description_width": "initial"},
    layout=widgets.Layout(width="95%"),
)

top_k_slider = widgets.IntSlider(
    value=5,
    min=0,
    max=200,
    step=5,
    description="Top-k:",
    style={"description_width": "initial"},
    layout=widgets.Layout(width="95%"),
)

generate_button = widgets.Button(
    description="Generate",
    button_style="primary",
    icon="play",
)

output_area = Output()

def generate_comparison(button):
    """Generate outputs from both models and display side-by-side."""
    if config is None:
        with output_area:
            print("\u26a0 Please configure backend and stage first")
        return
    
    with output_area:
        output_area.clear_output()
        
        idx = example_selector.value
        example = examples[idx]
        
        print(f"Generating for: {example['id']}")
        print("=" * 80)
        
        # Build prompt
        user_prompt = build_user_prompt(example["problem"])
        
        # Use the user-editable system prompt from config
        active_system_prompt = config.get("system_prompt", config["stage_config"].system_prompt)
        
        # Generation parameters
        gen_kwargs = dict(
            system_prompt=active_system_prompt,
            max_new_tokens=max_tokens_slider.value,
            temperature=temperature_slider.value,
            top_p=top_p_slider.value,
            top_k=top_k_slider.value,
        )
        
        # Resolve display labels from config
        base_label = config.get("base_label", config["stage_config"].base_model_id)
        tuned_label = config.get("tuned_label", config["stage_config"].tuned_model_id)
        
        # Generate from base model (if configured)
        base_output = None
        if config.get("base_backend"):
            print(f"\n[Base Model: {base_label}] Generating (max_tokens={gen_kwargs['max_new_tokens']}, "
                  f"temp={gen_kwargs['temperature']}, top_p={gen_kwargs['top_p']}, top_k={gen_kwargs['top_k']})...")
            try:
                base_output = config["base_backend"].generate(user_prompt, **gen_kwargs)
            except Exception as e:
                base_output = f"Error: {e}"
        
        # Generate from tuned model (if configured)
        tuned_output = None
        if config.get("tuned_backend"):
            print(f"[Tuned Model: {tuned_label}] Generating...")
            try:
                tuned_output = config["tuned_backend"].generate(user_prompt, **gen_kwargs)
            except Exception as e:
                tuned_output = f"Error: {e}"
        
        # Display side-by-side (or single panel if only one model)
        display(HTML("""
        <style>
        .comparison-container { display: flex; gap: 20px; }
        .model-output { flex: 1; border: 1px solid #ccc; padding: 10px; border-radius: 5px; }
        .base-model { background-color: #fff3cd; }
        .tuned-model { background-color: #d1ecf1; }
        </style>
        """))
        
        parts = []
        if base_output is not None:
            parts.append(f"""
            <div class="model-output base-model">
                <h3>Base Model ({base_label})</h3>
                <pre style="white-space: pre-wrap;">{base_output}</pre>
            </div>""")
        if tuned_output is not None:
            parts.append(f"""
            <div class="model-output tuned-model">
                <h3>Tuned Model ({tuned_label})</h3>
                <pre style="white-space: pre-wrap;">{tuned_output}</pre>
            </div>""")
        
        if parts:
            display(HTML(f'<div class="comparison-container">{"".join(parts)}</div>'))
        else:
            display(HTML("<p style='color:orange;'>\u26a0 No models configured for comparison.</p>"))
        
        # Store outputs for analysis (use empty string if model was not run)
        global last_base_output, last_tuned_output
        last_base_output = base_output or ""
        last_tuned_output = tuned_output or ""

generate_button.on_click(generate_comparison)

display(VBox([
    widgets.HTML("<h3>Generation Controls</h3>"),
    example_selector,
    example_preview_output,
    widgets.HTML("<b>Hyperparameters</b> (adjust for your hardware \u2014 larger tokens = longer output):"),
    max_tokens_slider,
    temperature_slider,
    top_p_slider,
    top_k_slider,
    generate_button,
    output_area,
]))

In [8]:
#@title Phase highlighting function (double-click to show code)
# Phase highlighting function
def highlight_phases(text: str) -> str:
    """Add HTML highlighting to Nyaya phases."""
    phases = [
        (r"## Samshaya.*?\n", "#ffeb3b"),
        (r"## Pramana.*?\n", "#4caf50"),
        (r"## Pancha Avayava.*?\n", "#2196f3"),
        (r"## Tarka.*?\n", "#ff9800"),
        (r"## Hetvabhasa.*?\n", "#9c27b0"),
        (r"## Nirnaya.*?\n", "#f44336"),
    ]
    
    for pattern, color in phases:
        text = re.sub(
            pattern,
            f'<span style="background-color: {color}; padding: 2px 4px; border-radius: 3px; font-weight: bold;">\\g<0></span>',
            text,
            flags=re.IGNORECASE,
        )
    
    return text

# Display highlighted output
if 'last_base_output' in globals():
    display(HTML(f"<h3>Base Model Output (Highlighted)</h3><pre>{highlight_phases(last_base_output)}</pre>"))
if 'last_tuned_output' in globals():
    display(HTML(f"<h3>Tuned Model Output (Highlighted)</h3><pre>{highlight_phases(last_tuned_output)}</pre>"))

## 4. Interactive Learning Exercises

In [16]:
#@title Exercise 1: Identify the doubt type (double-click to show code)
# Exercise 1: Identify the doubt type
exercise1_problem = """
Problem: If it rains, the ground gets wet. The ground is wet. Did it rain?
"""

exercise1_question = widgets.HTML(
    value="""
    <h4>Exercise 1: Doubt Type Identification</h4>
    <p>What type of doubt (Samshaya) is present in this problem?</p>
    <pre>{exercise1_problem}</pre>
    """.format(exercise1_problem=exercise1_problem)
)

exercise1_answer = widgets.Dropdown(
    options=[
        ("Select...", ""),
        ("Samana Dharma Upapatti", "samana_dharma_upapatti"),
        ("Vipratipatti", "vipratipatti"),
        ("Anadhyavasaya", "anadhyavasaya"),
    ],
    description="Answer:",
)

exercise1_feedback = Output()

def check_exercise1(change):
    with exercise1_feedback:
        exercise1_feedback.clear_output()
        if exercise1_answer.value == "vipratipatti":
            display(HTML("<p style='color: green;'>✓ Correct! This is Vipratipatti (conflicting possibilities) - rain could cause wet ground, but so could other things.</p>"))
        elif exercise1_answer.value:
            display(HTML("<p style='color: red;'>✗ Not quite. Think about whether there are conflicting possible explanations for the wet ground.</p>"))

exercise1_answer.observe(check_exercise1, names='value')

display(VBox([exercise1_question, exercise1_answer, exercise1_feedback]))

In [ ]:
#@title Exercise 2: Identify Pramana sources (double-click to show code)
# Exercise 2: Identify Pramana sources
exercise2_text = """
In a logic puzzle: "Alice says she has the red ball. Bob says Alice is lying."

What type of Pramana is "Alice says she has the red ball"?
"""

exercise2_question = widgets.HTML(
    value=f"""
    <h4>Exercise 2: Pramana Source Identification</h4>
    <p>{exercise2_text}</p>
    """.format(exercise2_text=exercise2_text)
)

exercise2_answer = widgets.Dropdown(
    options=[
        ("Select...", ""),
        ("Pratyaksha (Direct Perception)", "pratyaksha"),
        ("Anumana (Inference)", "anumana"),
        ("Upamana (Comparison)", "upamana"),
        ("Shabda (Testimony)", "shabda"),
    ],
    description="Answer:",
)

exercise2_feedback = Output()

def check_exercise2(change):
    with exercise2_feedback:
        exercise2_feedback.clear_output()
        if exercise2_answer.value == "shabda":
            display(HTML("<p style='color: green;'>✓ Correct! This is Shabda (testimony) - we are told what Alice said, not what we directly observed.</p>"))
        elif exercise2_answer.value:
            display(HTML("<p style='color: red;'>✗ Not quite. Think about whether this is something we directly observed or something we were told.</p>"))

exercise2_answer.observe(check_exercise2, names='value')

display(VBox([exercise2_question, exercise2_answer, exercise2_feedback]))

In [ ]:
#@title Exercise 3: Complete the syllogism (double-click to show code)
# Exercise 3: Complete the syllogism
exercise3_text = """
Complete this Pancha Avayava syllogism:

Pratijna (Thesis): All birds can fly.
Hetu (Reason): Because they have wings.
Udaharana (Universal Example): ?
Upanaya (Application): ?
Nigamana (Conclusion): ?
"""

exercise3_question = widgets.HTML(
    value=f"""
    <h4>Exercise 3: Complete the Syllogism</h4>
    <pre>{exercise3_text}</pre>
    """.format(exercise3_text=exercise3_text)
)

exercise3_udaharana = widgets.Textarea(
    value="",
    placeholder="Enter Udaharana (Universal Example)...",
    description="Udaharana:",
    layout=widgets.Layout(width="100%", height="80px"),
)

exercise3_feedback = Output()

def check_exercise3(change):
    with exercise3_feedback:
        exercise3_feedback.clear_output()
        answer = exercise3_udaharana.value.lower()
        
        # Check for universal rule pattern
        has_universal = any(word in answer for word in ["wherever", "whenever", "all", "any", "every"])
        has_example = any(word in answer for word in ["eagle", "sparrow", "bird", "example", "instance"])
        
        if has_universal and has_example:
            display(HTML("<p style='color: green;'>✓ Good! Your Udaharana includes both a universal rule and a concrete example. Example: 'Wherever there is a bird with wings, it can fly. For example, an eagle has wings and can fly.'</p>"))
        elif has_universal:
            display(HTML("<p style='color: orange;'>⚠ You have a universal rule, but try to include a concrete example too (e.g., 'like an eagle').</p>"))
        else:
            display(HTML("<p style='color: red;'>✗ Try to include both a universal rule (using words like 'wherever', 'whenever', 'all') and a concrete example.</p>"))

exercise3_udaharana.observe(check_exercise3, names='value')

display(VBox([exercise3_question, exercise3_udaharana, exercise3_feedback]))

## 5. Try Your Own Problem

In [ ]:
#@title Try your own problem (double-click to show code)
problem_input = widgets.Textarea(
    value="",
    placeholder="Enter your logical problem here...",
    description="Problem:",
    layout=widgets.Layout(width="100%", height="150px"),
)

custom_generate_button = widgets.Button(
    description="Generate Nyaya Reasoning",
    button_style="success",
    icon="rocket",
)

custom_output_area = Output()

def generate_custom(button):
    """Generate reasoning for custom problem using tuned model (fallback: base)."""
    if config is None:
        with custom_output_area:
            print("\u26a0 Please configure backend and stage first")
        return

    problem = problem_input.value.strip()
    if not problem:
        with custom_output_area:
            print("\u26a0 Please enter a problem")
        return

    with custom_output_area:
        custom_output_area.clear_output()

        backend = config.get("tuned_backend") or config.get("base_backend")
        if backend is None:
            print("\u26a0 No model backend available. Please apply configuration first.")
            return

        if config.get("tuned_backend"):
            model_label = config.get("tuned_label", config["stage_config"].tuned_model_id)
            model_role = "Tuned"
        else:
            model_label = config.get("base_label", config["stage_config"].base_model_id)
            model_role = "Base (tuned model not available)"

        print(f"Generating reasoning with {model_role} model: {model_label}")
        print("=" * 80)

        user_prompt = build_user_prompt(problem)
        active_system_prompt = config.get("system_prompt", config["stage_config"].system_prompt)

        try:
            output = backend.generate(
                user_prompt,
                system_prompt=active_system_prompt,
                max_new_tokens=max_tokens_slider.value if 'max_tokens_slider' in globals() else 2048,
                temperature=temperature_slider.value if 'temperature_slider' in globals() else 0.5,
                top_p=top_p_slider.value if 'top_p_slider' in globals() else 0.75,
                top_k=top_k_slider.value if 'top_k_slider' in globals() else 5,
            )
            display(Markdown(f"## Generated Reasoning ({model_role}: {model_label})\n\n{output}"))
        except Exception as e:
            display(HTML(f"<p style='color: red;'>Error: {e}</p>"))

custom_generate_button.on_click(generate_custom)

display(VBox([
    widgets.HTML("<h3>Try Your Own Problem</h3>"),
    problem_input,
    custom_generate_button,
    custom_output_area,
]))

---

## Summary

This notebook demonstrates:

1. **Structured Reasoning**: The 6-phase Navya-Nyaya methodology
2. **Model Comparison**: Side-by-side comparison of base vs tuned models
3. **Interactive Learning**: Exercises to understand Nyaya concepts
4. **Custom Problems**: Generate reasoning for your own logical problems